# План выполнения

In [3]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName("Read Session")
         .master("spark://spark-master:7077") 
         .config("spark.jars", 
                 "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar,"
                 "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar,"
                 "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar")
         .getOrCreate()
        )

25/10/29 18:14:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Показать план выполнения

In [9]:
df = spark.read.csv("/shared_data/bmw.csv", header=True, inferSchema=True)
df.explain("formatted")

[Stage 3:>                                                          (0 + 1) / 1]

== Physical Plan ==
Scan csv  (1)


(1) Scan csv 
Output [11]: [Model#56, Year#57, Region#58, Color#59, Fuel_Type#60, Transmission#61, Engine_Size_L#62, Mileage_KM#63, Price_USD#64, Sales_Volume#65, Sales_Classification#66]
Batched: false
Location: InMemoryFileIndex [file:/shared_data/bmw.csv]
ReadSchema: struct<Model:string,Year:int,Region:string,Color:string,Fuel_Type:string,Transmission:string,Engine_Size_L:double,Mileage_KM:int,Price_USD:int,Sales_Volume:int,Sales_Classification:string>




## Все варианты

In [8]:
# Или с указанием режима
df.explain()           # по умолчанию - простой режим
df.explain("simple")   # только физический план
df.explain("extended") # логический + физический план
df.explain("codegen")  # сгенерированный код
df.explain("cost")     # с стоимостным анализом (если доступно)
df.explain("formatted") # форматированный вывод

== Physical Plan ==
FileScan csv [Model#17,Year#18,Region#19,Color#20,Fuel_Type#21,Transmission#22,Engine_Size_L#23,Mileage_KM#24,Price_USD#25,Sales_Volume#26,Sales_Classification#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/shared_data/bmw.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Model:string,Year:int,Region:string,Color:string,Fuel_Type:string,Transmission:string,Engi...


== Physical Plan ==
FileScan csv [Model#17,Year#18,Region#19,Color#20,Fuel_Type#21,Transmission#22,Engine_Size_L#23,Mileage_KM#24,Price_USD#25,Sales_Volume#26,Sales_Classification#27] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/shared_data/bmw.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Model:string,Year:int,Region:string,Color:string,Fuel_Type:string,Transmission:string,Engi...


== Parsed Logical Plan ==
Relation [Model#17,Year#18,Region#19,Color#20,Fuel_Type#21,Transmission#

# Описание

В плане выполнения обычно смотрят **критические точки производительности** и **потенциальные проблемы**. Вот основные аспекты:

## 1. **Операции Shuffle (Exchange)**

```python
df.groupBy("region").agg(F.avg("price").alias("avg_price"))
```
В плане ищите:
```
*(2) HashAggregate(keys=[region#25], functions=[avg(price#27)])
+- Exchange hashpartitioning(region#25, 200)
   +- *(1) HashAggregate(keys=[region#25], functions=[partial_avg(price#27)])
```

**Что смотреть:** `Exchange` означает перераспределение данных между узлами - это **дорогая операция**!

## 2. **Pushed Filters vs Post-Read Filters**

```python
df.filter(F.col("year") > 2020).filter(F.col("price") > 50000)
```

В плане:
```
PushedFilters: [GreaterThan(year,2020), GreaterThan(price,50000)], 
```
**Что смотреть:** 
- `PushedFilters` - фильтры применены при чтении (хорошо ✅)
- Если фильтры в `Filter()` после `FileScan` - данные фильтруются после чтения (плохо ❌)

## 3. **Типы Join**

```python
df1.join(df2, "id", "inner")
```

В плане ищите:
```
*(5) SortMergeJoin [id#10], [id#20], Inner
```
**Что смотреть:**
- `BroadcastHashJoin` - оптимально для маленьких таблиц ✅
- `SortMergeJoin` - для больших таблиц
- `ShuffledHashJoin` - может быть дорогим

## 4. **Data Skew (Перекос данных)**

```python
df.groupBy("category").count()
```
В плане смотрите размеры партиций:
```
+- Exchange hashpartitioning(category#15, 200)
```
**Что смотреть:** если одна партиция обрабатывается значительно дольше - есть перекос данных

## 5. **Predicate Pushdown**

```python
df.filter(F.col("year") > 2020).select("model", "price")
```

В плане:
```
FileScan csv [model#17,price#25] 
PushedFilters: [GreaterThan(year,2020)]
```
**Что смотреть:** фильтры и проекции применены на уровне чтения ✅

## 6. **Количество партиций**

```python
df.rdd.getNumPartitions()  # можно проверить отдельно
```

В плане смотрите:
```
Exchange hashpartitioning(region#19, 200)  # 200 партиций
```

## 7. **Полное сканирование vs Прунинг партиций**

Для партиционированных данных:
```
PartitionFilters: [isnotnull(region#19), (region#19 = Europe)]
PushedFilters: [GreaterThan(price,50000)]
```
**Что смотреть:** 
- `PartitionFilters` - читаются только нужные партиции ✅
- Если `PartitionFilters: []` - читаются все партиции ❌

## 8. **Вложенные операции vs Стадиинг**

```python
# Плохо - два отдельных shuffle
df1.groupBy("a").agg(F.sum("b"))
df2.groupBy("a").agg(F.avg("c"))
result = df1.join(df2, "a")

# Лучше - один shuffle
df1_with_agg = df1.groupBy("a").agg(F.sum("b").alias("sum_b"))
df2_with_agg = df2.groupBy("a").agg(F.avg("c").alias("avg_c"))
result = df1_with_agg.join(df2_with_agg, "a")
```

## Практический пример анализа:

```python
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder \
    .appName("PlanAnalysis") \
    .master("local[*]") \
    .getOrCreate()

# Читаем данные
df = spark.read.csv("/shared_data/bmw.csv", header=True, inferSchema=True)

# Сложный запрос для анализа
complex_query = df \
    .filter(F.col("Year") > 2018) \
    .filter(F.col("Price_USD") > 30000) \
    .groupBy("Region", "Fuel_Type") \
    .agg(
        F.avg("Price_USD").alias("AvgPrice"),
        F.count("*").alias("Count"),
        F.max("Mileage_KM").alias("MaxMileage")
    ) \
    .join(
        df.groupBy("Region").agg(F.avg("Price_USD").alias("RegionAvgPrice")),
        "Region"
    ) \
    .filter(F.col("AvgPrice") > F.col("RegionAvgPrice")) \
    .orderBy("AvgPrice", ascending=False)

print("=== ANALYZING CRITICAL POINTS ===")
complex_query.explain("formatted")

# Анализируем ключевые аспекты:
print("\n=== KEY ANALYSIS POINTS ===")
print("1. Check for Exchange operations (shuffle)")
print("2. Check PushedFilters vs runtime Filters") 
print("3. Check Join type")
print("4. Check if aggregations are combined")
print("5. Check partition pruning")

spark.stop()
```

## Чеклист для анализа плана:

✅ **Хорошие признаки:**
- `PushedFilters` содержат ваши условия WHERE
- `BroadcastJoin` для маленьких таблиц
- Минимальное количество `Exchange` операций
- `PartitionFilters` используются для партиционированных данных
- Агрегации объединены где возможно

❌ **Проблемные признаки:**
- Много операций `Exchange` (shuffle)
- `Filter` операции после `FileScan` (не pushed)
- `CartesianProduct` для больших таблиц
- Отсутствие `PartitionFilters` для партиционированных данных
- Очень большое или маленькое количество партиций

## Как использовать эту информацию:

1. **Увидели много shuffle** → попробуйте переписать запрос
2. **Фильтры не pushed** → проверьте условия, измените порядок операций  
3. **Неоптимальный join** → используйте broadcast для маленьких таблиц
4. **Перекос данных** → добавьте salt или измените ключ группировки

Анализ плана выполнения - это ключ к оптимизации Spark приложений!